CITS4012 project 2

Authors: Theoridho Andily [22764884] & 

# Data Exploration

In [3]:
from gensim.corpora import Dictionary
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import gensim.downloader as api
import pandas as pd
import numpy as np
import gensim
import re

In [34]:
# Download a pretrained word2vec vectors from Google News dataset
wv = api.load('word2vec-google-news-300')

In [4]:
job_description = pd.read_csv('seek_australia.csv', usecols=[5])

In [6]:
# Select the first 2000 rows to simplify training
job_description = job_description.job_description[:2000].str.cat(sep=' ')

In [7]:
# Tokenise the document using gensim
doc_tokenized = gensim.utils.simple_preprocess(job_description, deacc=True)

In [20]:
def convert(tup, dict):
    """ This function takes a list of tuple and a dictionary to fill the dictionary"""
    for a, b in tup:
        dict[a] = b
    return dict
BoW_corpus_dict = dict()
BoW_dict = convert(BoW_corpus, BoW_corpus_dict)  

In [19]:
# Create a dictionary of {(word: frequency)}, the inner curved bracket indicates that the data is a tuple
dictionary = Dictionary()
BoW_corpus = dictionary.doc2bow(doc_tokenized, allow_update=True)
BoW_corpus = [(dictionary[id], freq) for id, freq in BoW_corpus]
# BoW_corpus[:10]
BoW_corpus_dict = dict()
BoW_dict = convert(BoW_corpus, BoW_corpus_dict) 

In [10]:
# Initiliasize the parameters for our own word2vec model using all cores
cores = 16
model = Word2Vec(min_count=1,
                     window=2,
                     vector_size=100,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [11]:
from time import time

t = time()

model.build_vocab([doc_tokenized], progress_per=10)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.01 mins


In [12]:
t = time()

model.train(doc_tokenized, total_examples=model.corpus_count, epochs=1000, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 13.69 mins


In [17]:
#model.wv.key_to_index.keys()

In [23]:
# find tsne coords for 2 dimensions
tsne = TSNE(n_components=2, random_state=0)

In [27]:
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
import pandas as pd

In [28]:
def interactive_tsne(text_labels, tsne_array):
    '''makes an interactive scatter plot with text labels for each point'''

    # Define a dataframe to be used by bokeh context
    bokeh_df = pd.DataFrame(tsne_array, text_labels, columns=['x','y'])
    bokeh_df['text_labels'] = bokeh_df.index

    # interactive controls to include to the plot
    TOOLS="hover, zoom_in, zoom_out, box_zoom, undo, redo, reset, box_select"

    p = figure(tools=TOOLS, plot_width=700, plot_height=700)

    # define data source for the plot
    source = ColumnDataSource(bokeh_df)

    # scatter plot
    p.scatter('x', 'y', source=source, fill_alpha=0.6,
              fill_color="#8724B5",
              line_color=None)

    # text labels
    labels = LabelSet(x='x', y='y', text='text_labels', y_offset=8,
                      text_font_size="8pt", text_color="#555555",
                      source=source, text_align='center')

    p.add_layout(labels)

    # show plot inline
    output_notebook()
    show(p)

In [29]:
# NO IDEA WHAT IS GOING ON HERE
import nltk
from nltk.corpus import stopwords

vocab_sorted = dict(sorted(BoW_corpus_dict.items(), key=lambda item: item[1], reverse=True))
#print(vocab_sorted)

stopwords = stopwords.words('english')
input_vocab =  [word for word in vocab_sorted if word in model.wv.key_to_index.keys() and word not in stopwords]
points = len(input_vocab)
X = model.wv[input_vocab]
X_tsne = tsne.fit_transform(X[:points])

#print(input_vocab)

#points = len(input_vocab)

interactive_tsne(list(input_vocab)[:points], X_tsne)

c:\envs\cits4012\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\envs\cits4012\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


Loading BokehJS ...

In [30]:
# Our trained domain specific embeddings
model.wv.most_similar(positive=["language"])

[('settera', 0.3949071764945984),
 ('scrap', 0.37574535608291626),
 ('octopus', 0.36116334795951843),
 ('triage', 0.3402031362056732),
 ('wishes', 0.3356758952140808),
 ('aig', 0.33101245760917664),
 ('ehealth', 0.32696935534477234),
 ('worth', 0.324599951505661),
 ('archdiocese', 0.3230147957801819),
 ('gladstone', 0.32281026244163513)]

In [33]:
# Pretrained embeddings
wv.most_similar(positive=["language"])

[('langauge', 0.747669517993927),
 ('Language', 0.6695358157157898),
 ('languages', 0.6341331601142883),
 ('English', 0.6120712757110596),
 ('CMPB_Spanish', 0.6083105802536011),
 ('nonnative_speakers', 0.6063110828399658),
 ('idiomatic_expressions', 0.5889802575111389),
 ('verb_tenses', 0.5841568112373352),
 ('Kumeyaay_Diegueno', 0.5798824429512024),
 ('dialect', 0.5724599957466125)]